# A neural network approach to privacy preservingmulti-engagement prediction on Twitter - code

This notebook contains all the results presented in our paper : "A neural network approach to privacy preservingmulti-engagement prediction on Twitter"

In [1]:
#### requirements

%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd
import collections
import re
import pprint as pp
import numpy as np
import collections

import multiprocessing as mp
from multiprocessing.pool import ThreadPool

import math
import gzip
import pickle as pkl
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer 

import fonctions
from batch_processing import *
import metric_loading

import itertools
from datetime import datetime
from tqdm.notebook import tqdm

import os
from os import listdir
from os.path import isfile, join

import random
random.seed(0)



Using TensorFlow backend.


# Section 2 : Dataset Characteristics

## Section 2.1 : Severe imbalance

In [ ]:
user_ratio = metric_loading.get_global_user_ratio2(True)

In [ ]:
# library
import matplotlib.pyplot as plt
 
# Create bars

nbrs= somme3
lbl=['LK', 'RTC', 'LK+RT','RT','LK+RTC','RPL','LK+RPL','RT+RPL','LK+RT+RPL','LK+RTC+RPL','RTC+RPL']
dico = dict(zip(lbl,nbrs))

dico = {k: v for k, v in sorted(dico.items(),reverse=True, key=lambda item: item[1])}

bars = list(dico.values())
 
# The X position of bars

r = range(11)
 
# Create barplot
plt.bar(r, bars, color = (0.3,0.5,0.4,0.6) ) #width = barWidth, 
 
# Text below each barplot with a rotation at 90°
plt.xticks([r for r in range(len(r))], dico.keys(), rotation=90)
 
# Create labels
label = [str(x)+'%' for x in dico.values() ]
 
# Text on the top of each barplot
for i in range(len(r)):
    plt.text(x = r[i]-0.4 , y = bars[i]+0.5, s = label[i], size = 8)
    
plt.xlabel('Engagement type')
plt.ylabel('Frequency')

# Adjust the margins
#plt.subplots_adjust(bottom= 0.1, top = 0.98)
plt.savefig('/home/maxime/Desktop/RecSys2020/engagement_distribution_relat.png', bbox_inches='tight')

# Show graphic
plt.show()

In [ ]:
# library
import matplotlib.pyplot as plt
 
# Create bars

dico = {'LK':34.8, 'RTC':0.44, 
              'LK+RT': 4.73,'RT':4.56,
              'LK+RTC': 0.28,'RPL':1.66,
              'LK+RPL':0.83,'RT+RPL': 0.05,
              'LK+RT+RPL': 0.14,'LK+RTC+RPL':0.002,'RTC+RPL':0.002}
dico = {k: v for k, v in sorted(dico.items(),reverse=True, key=lambda item: item[1])}

bars = list(dico.values())
 
# The X position of bars

r = range(11)
 
# Create barplot
plt.bar(r, bars, color = (0.3,0.5,0.4,0.6) ) #width = barWidth, 
 
# Text below each barplot with a rotation at 90°
plt.xticks([r for r in range(len(r))], dico.keys(), rotation=90)
 
# Create labels
label = [str(x)+'%' for x in dico.values() ]
 
# Text on the top of each barplot
for i in range(len(r)):
    plt.text(x = r[i]-0.4 , y = bars[i]+0.5, s = label[i], size = 8)
    
plt.xlabel('Engagement type')
plt.ylabel('Frequency')

# Adjust the margins
#plt.subplots_adjust(bottom= 0.1, top = 0.98)
plt.savefig('/home/maxime/Desktop/RecSys2020/engagement_distribution.png', bbox_inches='tight')

# Show graphic
plt.show()

## Section 2.2 : Different categories of users

In [ ]:
user_presence =  metric_loading.get_global_user_presence(False) 
user_ratio_t = metric_loading.get_global_user_ratio2(True)
print(len(user_presence))
print(len(user_ratio_t))
overlap = [ k for k in user_presence if k in user_ratio_t]
print( len(overlap) )

update_user_ratio = { k:user_ratio_t[k] for k,v in tqdm(user_presence.items() ) if k in user_ratio_t }

print( len(update_user_ratio.keys()) )

In [ ]:
1873604/6561475

# Section 3 : Experiments

### Fold1

In [15]:
### base

lk = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/like_timestamp_1000.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rt = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/retweet_timestamp_1000.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rtc = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/retweet_with_comment_timestamp_1000.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rpl = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/reply_timestamp_1000.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)

print( fonctions.compute_rce(lk[6], lk[2]) )
print( fonctions.compute_prauc(lk[6], lk[2]) )

print( fonctions.compute_rce(rt[6], rt[3]) )
print( fonctions.compute_prauc(rt[6], rt[3]) )

print( fonctions.compute_rce(rtc[6], rtc[4]) )
print( fonctions.compute_prauc(rtc[6], rtc[4]) )

print( fonctions.compute_rce(rpl[6], rpl[5]) )
print( fonctions.compute_prauc(rpl[6], rpl[5]) )

11.364470590608933
0.6700475605513339
6.751337284120817
0.23058123604529718
1.2399614780955526
0.01212750341245822
7.0983653093707915
0.07319192510232156


In [5]:
### double base + custom loss

lk = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_like_timestamp_4000.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rt = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_retweet_timestamp_4000.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rtc = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_retweet_with_comment_timestamp_4000.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rpl = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_reply_timestamp_4000.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)

print( fonctions.compute_rce(lk[2], lk[3]) )
print( fonctions.compute_prauc(lk[2], lk[3]) )

print( fonctions.compute_rce(rt[2], rt[3]) )
print( fonctions.compute_prauc(rt[2], rt[3]) )

print( fonctions.compute_rce(rtc[2], rtc[3]) )
print( fonctions.compute_prauc(rtc[2], rtc[3]) )

print( fonctions.compute_rce(rpl[2], rpl[3]) )
print( fonctions.compute_prauc(rpl[2], rpl[3]) )

17.419420202918158
0.7446501613791754
17.354942866994026
0.43686925630508155
-10.17001427235833
0.030349176090832777
9.076041978703353
0.10294857627384545


In [29]:
### double base + binary cross entropy

lk = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_like_timestamp_3000.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rt = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_retweet_timestamp_3000.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rtc = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_retweet_with_comment_timestamp_3000.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rpl = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_reply_timestamp_3000.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)

print( fonctions.compute_rce(lk[2], lk[3]) )
print( fonctions.compute_prauc(lk[2], lk[3]) )

print( fonctions.compute_rce(rt[2], rt[3]) )
print( fonctions.compute_prauc(rt[2], rt[3]) )

print( fonctions.compute_rce(rtc[2], rtc[3]) )
print( fonctions.compute_prauc(rtc[2], rtc[3]) )

print( fonctions.compute_rce(rpl[2], rpl[3]) )
print( fonctions.compute_prauc(rpl[2], rpl[3]) )

15.48973380522879
0.7437274951516486
17.522200208406137
0.4458199404488744
-0.6129504690818299
0.033356791279591486
7.356574947434247
0.10371663176339563


In [4]:
### double + all

lk = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_like_timestamp_5000.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rt = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_retweet_timestamp_5000.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rtc = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_retweet_with_comment_timestamp_5000.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rpl = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_reply_timestamp_5000.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)

print( fonctions.compute_rce(lk[2], lk[3]) )
print( fonctions.compute_prauc(lk[2], lk[3]) )

print( fonctions.compute_rce(rt[2], rt[3]) )
print( fonctions.compute_prauc(rt[2], rt[3]) )

print( fonctions.compute_rce(rtc[2], rtc[3]) )
print( fonctions.compute_prauc(rtc[2], rtc[3]) )

print( fonctions.compute_rce(rpl[2], rpl[3]) )
print( fonctions.compute_prauc(rpl[2], rpl[3]) )

16.836352848593407
0.7470348249219674
17.029411306229324
0.4474760717060683
-1.6307659964126309
0.03079751103784525
7.169041032124301
0.10727302937750045


### Fold 2

In [2]:
### base

lk = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/like_timestamp_1001.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rt = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/retweet_timestamp_1001.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rtc = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/retweet_with_comment_timestamp_1001.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rpl = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/reply_timestamp_1001.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)

print( fonctions.compute_rce(lk[6], lk[2]) )
print( fonctions.compute_prauc(lk[6], lk[2]) )

print( fonctions.compute_rce(rt[6], rt[3]) )
print( fonctions.compute_prauc(rt[6], rt[3]) )

print( fonctions.compute_rce(rtc[6], rtc[4]) )
print( fonctions.compute_prauc(rtc[6], rtc[4]) )

print( fonctions.compute_rce(rpl[6], rpl[5]) )
print( fonctions.compute_prauc(rpl[6], rpl[5]) )

10.905607511416559
0.6714551937511261
6.57256197711289
0.23069437899488698
0.7856290950647105
0.01166535696056791
7.378762435961894
0.06901068294829077


In [7]:
## double base

lk = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_like_timestamp_3001.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rt = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_retweet_timestamp_3001.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rtc = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_retweet_with_comment_timestamp_3001.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rpl = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_reply_timestamp_3001.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)

print( fonctions.compute_rce(lk[2], lk[3]) )
print( fonctions.compute_prauc(lk[2], lk[3]) )

print( fonctions.compute_rce(rt[2], rt[3]) )
print( fonctions.compute_prauc(rt[2], rt[3]) )

print( fonctions.compute_rce(rtc[2], rtc[3]) )
print( fonctions.compute_prauc(rtc[2], rtc[3]) )

print( fonctions.compute_rce(rpl[2], rpl[3]) )
print( fonctions.compute_prauc(rpl[2], rpl[3]) )

14.820905224255121
0.7479255191231939
17.657765506474433
0.4439929634861524
-0.468292820276317
0.03335136814750482
6.607245486830548
0.10233268394182321


In [6]:
### double base + custom loss

lk = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_like_timestamp_4001.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rt = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_retweet_timestamp_4001.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rtc = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_retweet_with_comment_timestamp_4001.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rpl = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_reply_timestamp_4001.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)

print( fonctions.compute_rce(lk[2], lk[3]) )
print( fonctions.compute_prauc(lk[2], lk[3]) )

print( fonctions.compute_rce(rt[2], rt[3]) )
print( fonctions.compute_prauc(rt[2], rt[3]) )

print( fonctions.compute_rce(rtc[2], rtc[3]) )
print( fonctions.compute_prauc(rtc[2], rtc[3]) )

print( fonctions.compute_rce(rpl[2], rpl[3]) )
print( fonctions.compute_prauc(rpl[2], rpl[3]) )

17.387817068745747
0.747406788103898
17.41747630235477
0.44843326312659293
0.2386478664903935
0.03202701105658746
7.053214683615272
0.10443681982933425


In [5]:
### double + custom loss + new features

lk = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_like_timestamp_5001.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rt = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_retweet_timestamp_5001.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rtc = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_retweet_with_comment_timestamp_5001.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rpl = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_reply_timestamp_5001.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)

print( fonctions.compute_rce(lk[2], lk[3]) )
print( fonctions.compute_prauc(lk[2], lk[3]) )

print( fonctions.compute_rce(rt[2], rt[3]) )
print( fonctions.compute_prauc(rt[2], rt[3]) )

print( fonctions.compute_rce(rtc[2], rtc[3]) )
print( fonctions.compute_prauc(rtc[2], rtc[3]) )

print( fonctions.compute_rce(rpl[2], rpl[3]) )
print( fonctions.compute_prauc(rpl[2], rpl[3]) )

14.357551279834935
0.7482304342660203
12.717934492926709
0.44212294861521073
-8.906587311689718
0.029137764530494107
4.50160732542818
0.11207792934039895


### Efficiency of the custom loss

In [ ]:
### impact of the custom loss on the prediction

rt = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_retweet_timestamp_4000.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rtc = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_retweet_with_comment_timestamp_4000.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
table1 = pd.concat([ rt[2],rtc[2] ],axis=1)
table1.columns=['rt','rtc']

print( len( table1[ (table1['rtc']>0.5) & (table1['rt']<0.5) ] ) )
print( len( table1[ table1['rtc'] > table1['rt'] ] ) )

rt_custom = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_retweet_timestamp_3000.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rtc_custom = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_retweet_with_comment_timestamp_3000.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
table2 = pd.concat([rt_custom[2],rtc_custom[2]],axis=1)
table2.columns=['rt','rtc']

print( len( table2[ (table2['rtc']>0.5) & (table2['rt']<0.5) ] ) )
print( len( table2[ table2['rtc'] > table2['rt'] ]) )



In [9]:
rt = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_retweet_timestamp_0.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rtc = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_retweet_with_comment_timestamp_0.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
table1 = pd.concat([ rt[2],rtc[2] ],axis=1)
table1.columns=['rt','rtc']

print( len( table1[ (table1['rtc']>0.5) & (table1['rt']<0.5) ] ) )
print( len( table1[ table1['rtc'] > table1['rt'] ] ) )

rt_custom = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_retweet_timestamp_9999.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rtc_custom = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_retweet_with_comment_timestamp_9999.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
table2 = pd.concat([rt_custom[2],rtc_custom[2]],axis=1)
table2.columns=['rt','rtc']

print( len( table2[ (table2['rtc']>0.5) & (table2['rt']<0.5) ] ) )
print( len( table2[ table2['rtc'] > table2['rt'] ]) )



92
25405
0
0


### Fold 3 

In [2]:
### base

lk = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/like_timestamp_1005.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rt = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/retweet_timestamp_1005.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rtc = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/retweet_with_comment_timestamp_1005.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rpl = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/reply_timestamp_1005.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)

print( fonctions.compute_rce(lk[6], lk[2]) )
print( fonctions.compute_prauc(lk[6], lk[2]) )

print( fonctions.compute_rce(rt[6], rt[3]) )
print( fonctions.compute_prauc(rt[6], rt[3]) )

print( fonctions.compute_rce(rtc[6], rtc[4]) )
print( fonctions.compute_prauc(rtc[6], rtc[4]) )

print( fonctions.compute_rce(rpl[6], rpl[5]) )
print( fonctions.compute_prauc(rpl[6], rpl[5]) )

11.52398662998021
0.6710049318662645
6.854939678237704
0.2321925068878797
1.580620479956829
0.012142447731780366
7.884508097635578
0.07402883567820934


In [11]:
## double base

lk = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_like_timestamp_3005.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rt = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_retweet_timestamp_3005.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rtc = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_retweet_with_comment_timestamp_3005.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)
rpl = pd.read_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_reply_timestamp_3005.csv', encoding="utf-8", sep=',', header=None, index_col=False, low_memory=True)

print( fonctions.compute_rce(lk[2], lk[3]) )
print( fonctions.compute_prauc(lk[2], lk[3]) )

print( fonctions.compute_rce(rt[2], rt[3]) )
print( fonctions.compute_prauc(rt[2], rt[3]) )

print( fonctions.compute_rce(rtc[2], rtc[3]) )
print( fonctions.compute_prauc(rtc[2], rtc[3]) )

print( fonctions.compute_rce(rpl[2], rpl[3]) )
print( fonctions.compute_prauc(rpl[2], rpl[3]) )

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').